In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
import time

np.random.seed(777)

## Functions

In [2]:
DIV_CONST = 10

In [3]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [4]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [5]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
        
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
          
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [6]:
def get_SCLM_v100(melody_w_times_A, melody_w_times_B):
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    # Actual DP implementation
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            # If there is a match a possible solution is the previous plus one
            curr_value = 0
            
            tot_delta_time = (float(melody_w_times_A[i][1]) + float(melody_w_times_B[j][1])) / float(DIV_CONST)
            tot_diff_time = np.abs(float(melody_w_times_A[i][1]) - float(melody_w_times_B[j][1]))
            
            
            if (melody_w_times_A[i][3] == melody_w_times_B[j][3]) and (tot_diff_time <= tot_delta_time):
                curr_value = memo[i + 1][j + 1] + 1
                
            # The actual solution is the maximum between the one if there is a match, or skip on the melody A or melody B
            curr_value = max(curr_value, max(memo[i + 1][j], memo[i][j + 1]))
            
            # Save the solution
            memo[i][j] = curr_value
    
    # With the memoization table we can retrieve the actual melody
    i = 0
    j = 0
    SCLM = []
    while i != lim_A and j != lim_B:
    
        if ((memo[i + 1][j + 1] + 1) == memo[i][j]):
            SCLM.append((i, j))
            i += 1
            j += 1
        elif (memo[i + 1][j] == memo[i][j]):
            i += 1
        elif (memo[i][j + 1] == memo[i][j]):
            j += 1
    
    return SCLM

In [7]:
def get_max_timestamp_dif(melody_w_times_A, melody_w_times_B):
    return max(
        melody_w_times_A[len(melody_w_times_A) - 1][0] - melody_w_times_A[0][0],
        melody_w_times_B[len(melody_w_times_B) - 1][0] - melody_w_times_B[0][0]
    )

In [8]:
def getDifSCLM(melody_w_times_A, melody_w_times_B, sclm):
    
    # If there is no sclm or it is just one return max possible value
    if (len(sclm) <= 1):
        return get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    
    
    # Initialize the arrays
    T_A = np.zeros(shape=(len(sclm) - 1))
    T_B = np.zeros(shape=(len(sclm) - 1))
    T_C = np.zeros(shape=(len(sclm) - 1))
    Dif_ = np.zeros(shape=(len(sclm) - 1))
    
    for i in range(1, len(sclm)):
        T_A[i - 1] = melody_w_times_A[sclm[i][0]][0] - melody_w_times_A[sclm[i-1][0]][0]
        T_B[i - 1] = melody_w_times_B[sclm[i][1]][0] - melody_w_times_B[sclm[i-1][1]][0]
        T_C[i - 1] = np.abs(T_A[i - 1] - T_B[i - 1])
    
    T_C_mean = np.mean(T_C)
    
    for i in range(0, len(T_B)):
        T_B[i] += T_C_mean
        Dif_[i] = T_A[i] - T_B[i]
    
    return T_C_mean
    

In [9]:
def get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B):
    
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        print("EMPTY")
        return 1
    
    # Initialize result variable
    result_value = 0
    
    # Get Keys
    key_A = getSongKeyFromMelody_W_Times(melody_w_times_A)
    key_B = getSongKeyFromMelody_W_Times(melody_w_times_B)
    
    # D1: Scale  
    scale_dif1 = 0
    if (key_A.name != key_B.name):
        scale_dif1 = W1
    result_value += scale_dif1
    
    # D2: Mode  
    mode_dif2 = 0
    if (key_A.mode != key_B.mode):
        mode_dif2 = W2
    result_value += mode_dif2
    
    # Get SCLM v100
    sclm = get_SCLM_v100(melody_w_times_A, melody_w_times_B)
    
    # Get max len
    max_len = max(len(melody_w_times_A), len(melody_w_times_B))
    
    # D3: SCLM Length
    sclmlen_dif3 = ((max_len - len(sclm)) / max_len) * W3
    result_value += sclmlen_dif3
    
    # Get the Diff on temporal spacing in the SCLM
    dif_sclm = getDifSCLM(melody_w_times_A, melody_w_times_B, sclm)
    
    # D4: dif in sclm
    max_timestamp_dif = get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    sclmdif_dif4 = (dif_sclm / max_timestamp_dif) * W4
    result_value += sclmdif_dif4
    
    return result_value

## Traverse DATA

In [10]:
## NES ##
NOBUO_DATASET_PATH = "/Users/diego/unam/2021-I/aprendizaje_profundo/proyecto/nobuo/"

In [11]:
# Traverse midi files
nes_song_filenames = []
for root, directories, files in os.walk(NOBUO_DATASET_PATH):
    for file in files:
        nes_song_filenames.append(file)
print(nes_song_filenames[:3])
print(len(nes_song_filenames))

["FF6-_Devil's_Lab.mid", 'YGM - 4-04 A Secret, Sleeping in the Deep Sea.mid', "GM - 1-21 Red XIII's Theme.mid"]
1060


In [13]:
%%time

W1 = 0.0
W2 = 0.0
W3 = 1.0
W4 = 0.0

# Read Files 
MAX_LIM_NES_SONGS = len(nes_song_filenames)
len_nes_song_filenames = len(nes_song_filenames)
nes_songs_with_error = []
nes_similarities_for_sort = []


# Query File
# "322_SuperMarioBros__02_03SwimmingAround.mid"
# "322_SuperMarioBros__10_11SavedthePrincess.mid"
# "339_Tetris_00_01TitleScreen.mid"
song_filename_query = "007S-END.mid"
song_stream_query = m21.converter.parseFile(os.path.join(NOBUO_DATASET_PATH, song_filename_query))
midi_tracks_query = m21.midi.translate.streamToMidiFile(song_stream_query)
melody_w_times_query = getMelodyDeltaTimes(midi_tracks_query.tracks[1].events)

# We traverse the reduced table
cnt = 1
for song_filename_test in nes_song_filenames:
    try:
        
        song_stream_test = m21.converter.parseFile(os.path.join(NOBUO_DATASET_PATH, song_filename_test))
        midi_tracks_test = m21.midi.translate.streamToMidiFile(song_stream_test)
        melody_w_times_test = getMelodyDeltaTimes(midi_tracks_test.tracks[1].events)

        similarity_distance = get_MTRC_v100_from_melody_w_times(
            melody_w_times_query,
            melody_w_times_test)
        
        nes_similarities_for_sort.append((song_filename_test, similarity_distance))
        
        with open('./PROP2_query_to_sort_{0}.json'.format(song_filename_query.split(".")[0]), 'w') as outfile:
            json.dump({"data":nes_similarities_for_sort}, outfile)
        
    except:
        print("[ERROR!]")
        nes_songs_with_error.append(song_filename_test)
    finally:
        print("{0}/{1} - {2} - {3}".format(cnt, len_nes_song_filenames, song_filename_test, similarity_distance))
        cnt += 1
        if (cnt == MAX_LIM_NES_SONGS):
            break
    

 - TerraMix.mid - 1.0
672/1060 - Phantomf.mid - 1.0
673/1060 - ff9-02-11-at_the_south_gate_border.mid - 0.9696969696969697
674/1060 - XG - 3-23 Who Am I.mid - 0.9947916666666666
675/1060 - ff2lsub.mid - 0.9857142857142858
676/1060 - FF3Boss.mid - 0.9895833333333334
677/1060 - 064s-choco.mid - 0.9845679012345679
678/1060 - Ff6dance.mid - 1.0
679/1060 - ff9-01-22-gameover.mid - 0.7632508833922261
680/1060 - ff2babil.mid - 0.9114583333333334
681/1060 - 121-The Trial.mid - 0.9807692307692307
682/1060 - Ff4airsh.mid - 0.9866666666666667
683/1060 - YGM - 2-15 Conitnue.mid - 0.9947916666666666
684/1060 - 048s-horizon.mid - 1.0
685/1060 - YGM - 3-07 Cid's Theme.mid - 0.9895833333333334
686/1060 - YGM - 2-04 On That Day, 5 Years Ago.mid - 0.9947916666666666
687/1060 - XG - 1-04 Anxious Heart.mid - 1.0
688/1060 - GM - 2-18 Mining Town.mid - 1.0
689/1060 - celes.mid - 0.9895833333333334
690/1060 - GM - 1-06 Barett's Theme.mid - 0.9322916666666666
691/1060 - 099s-joriku2.mid - 0.9947916666666666
6

In [14]:
nes_similarities_for_sort

4796747967),
 ('FF6_-_Techno_de_chocobo.mid', 1.0),
 ('XG - 2-14 J-E-N-O-V-A.mid', 1.0),
 ('prlde.mid', 0.9947916666666666),
 ('magitek2.mid', 0.827485380116959),
 ('ff9-04-06-terra.mid', 0.8632075471698113),
 ("GM - 2-11 Rufus's Welcoming Ceremony.mid", 0.9066666666666666),
 ('figaro3.mid', 1.0),
 ('Ff2win.mid', 0.9902912621359223),
 ('YGM - 2-14 J-E-N-O-V-A.mid', 1.0),
 ('ff3over5.mid', 1.0),
 ('GM - 1-17 Who Are You.mid', 1.0),
 ('308-Undersea Palace.mid', 0.8729641693811075),
 ('ff9-02-09-ukele_le_chocobo.mid', 0.9360655737704918),
 ('setzer2.mid', 0.9429928741092637),
 ('GM - 2-04 On That Day, 5 Years Ago.mid', 0.9947916666666666),
 ('XG - 2-06 Waltz de Chocobo.mid', 0.9947916666666666),
 ('ff2under.mid', 0.984375),
 ('ff6final4.mid', 1.0),
 ('ff9-04-10-endless_sorrow.mid', 0.9903303787268332),
 ('AWE - 4-08 Hurry Faster!.mid', 0.9788135593220338),
 ('YGM - 4-14 Jenova Absolute.mid', 1.0),
 ('locke2.mid', 0.9403409090909091),
 ('055s-jail.mid', 1.0),
 ('ff3cyan_the.mid', 0.9791666

In [17]:
nes_similarities_for_sort.sort(key=lambda x: x[1])

In [18]:
nes_similarities_for_sort

4),
 ('XG - 4-07 On the Other Side of the Mountain.mid', 0.9895833333333334),
 ('114-Huh!.mid', 0.9895833333333334),
 ('069s-enzetu.mid', 0.9895833333333334),
 ('XG - 2-02 Ahead on Our Way.mid', 0.9895833333333334),
 ('3-07-SearchingForFriends-v1.1.mid', 0.9895833333333334),
 ('GM - 3-20 Buried in the Snow.mid', 0.9895833333333334),
 ('052s-travia.mid', 0.9895833333333334),
 ('206-The Day the World Revived.mid', 0.9895833333333334),
 ('AWE - 4-09 Sending a Dream Into the Universe.mid', 0.9895833333333334),
 ('214-Delightful Spekkio.mid', 0.9895833333333334),
 ('XG - 3-09 Wutai.mid', 0.9895833333333334),
 ('GM - 3-08 Steal the Tiny Bronco!.mid', 0.9895833333333334),
 ('AWE - 1-12 Flowers Blooming in the Chruch.mid', 0.9895833333333334),
 ('GM - 3-02 Life Stream.mid', 0.9895833333333334),
 ('NewContinent.mid', 0.9896907216494846),
 ('seal.mid', 0.9897540983606558),
 ('at-zero.mid', 0.9901356350184957),
 ("GM - 2-20 Cait Sith's Theme.mid", 0.9901477832512315),
 ("AWE - 2-20 Cait Sith's Th

In [19]:
with open('./PROP2_query_sorted_{0}.json'.format(song_filename_query.split(".")[0]), 'w') as outfile:
            json.dump({"data":nes_similarities_for_sort}, outfile)